# Surmortalité du coronavirus : préparation des données

Attention, changer les valeurs ci-dessous avant d'exécuter le notebook !

In [ ]:
folder_data = 'raw_data'  # dossier de téléchargement des données INSEE
folder_treated_data = 'treated_data'  # dossier d'enregistrement des données préparées
recompute_9819 = False   # Passer à False après la première exécution du notebook (étape très longue), utile pour recalculer seulement pour 2020

Sources : INSEE et Santé Publique France
+ Population (série longue) : https://www.insee.fr/fr/statistiques/4135499?sommaire=4136000
+ Fichier des décès : https://www.insee.fr/fr/information/4190491
+ Suivi extraordinaire des décès pour l'épidémie : https://www.insee.fr/fr/information/4470857
+ Population au 1er janvier 2018 : https://www.insee.fr/fr/statistiques/3303333?sommaire=3353488
+ Population au 1er janvier 2019 : https://www.insee.fr/fr/statistiques/3676587?sommaire=3696937
+ Population au 1er janvier 2020 : https://www.insee.fr/fr/statistiques/1892086?sommaire=1912926
+ Décès rapportés en milieu hospitalier et en EHPAD : https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/#resource-community-32c685b2-2ff8-44d6-a647-5df16cdaa6b1

In [ ]:
import os
import zipfile
import datetime
import sklearn
import bqplot
import pickle

import requests as rq
import numpy as np
import pandas as pd
import ipywidgets as ipw

from bqplot import pyplot as plt
from scipy import stats as scs
from numpy import random as npr

In [ ]:
try:
    os.makedirs(folder_data)
    to_download = True
except:
    to_download = False
    pass
os.makedirs(folder_treated_data, exist_ok=True)

def download(url):
    filename = url.split('/')[-1]
    path = os.path.join(folder_data, filename)
    res = rq.get(url, timeout=50)
    res.raise_for_status()
    with open(path, 'wb') as f:
        f.write(res.content)
    print(f'saved {path}')
    return path

def extract(path):
    with zipfile.ZipFile(path, 'r') as obj:
        obj.extractall(folder_data)
    print(f'extracted {path}')
        
def download_extract(url):
    path = download(url)
    extract(path)
    

In [ ]:
if to_download:
    urls = [
        'https://www.insee.fr/fr/statistiques/fichier/4190491/Deces_2020_M03.zip',
        'https://www.insee.fr/fr/statistiques/fichier/4190491/Deces_2020_M02.zip',
        'https://www.insee.fr/fr/statistiques/fichier/4190491/Deces_2020_M01.zip',
        'https://www.insee.fr/fr/statistiques/fichier/4190491/deces-2010-2018-csv.zip',
        'https://www.insee.fr/fr/statistiques/fichier/4190491/deces-2000-2009-csv.zip',
        'https://www.insee.fr/fr/statistiques/fichier/4190491/deces-1990-1999-csv.zip',
        'https://www.insee.fr/fr/statistiques/fichier/4135499/fe_t6.xls',
        'https://www.insee.fr/fr/statistiques/fichier/3676587/T19F032.xls',
        'https://www.insee.fr/fr/statistiques/fichier/1892086/pop-totale-france.xls',
        'https://www.insee.fr/fr/statistiques/fichier/3303333/T18F032.xls',
    ]

    for url in urls:
        download_extract(url)

In [ ]:
# Fichier JSON pour régions France
if to_download:
    url = 'https://www.pierremarion23.github.io/notebooks/france.json'
    download(url)

In [ ]:
# Données mises à jour régulièrement (une fois par semaine)
url = 'https://www.insee.fr/fr/statistiques/fichier/4470857/2020-04-24_detail.zip'
download_extract(url)

In [ ]:
url = 'https://www.data.gouv.fr/fr/datasets/r/32c685b2-2ff8-44d6-a647-5df16cdaa6b1'
download(url)
os.rename(os.path.join(folder_data, '32c685b2-2ff8-44d6-a647-5df16cdaa6b1'), os.path.join(folder_data, 'deces_hop_ehpad.csv'))

### Décès entre 1998 et 2019

On rajoute aussi les tables pour début 2020, mais seulement pour prendre en compte les décès de 2019 qui sont rapportés début 2020 (ce décalage peut qui peut arriver pour les décès en fin d'année). Comme cette étape est longue, on le fait seulement si on ne trouve pas déjà le fichier correspondant.

In [ ]:
def departement(s):
    try:
        if s[:2] == '97':
            return s[:3]
        return s[:2]
    except:
        return s

In [ ]:
if recompute_9819:
    
    # load data
    dfs_deces_raw = []
    for year in range(1998, 2020):
        path = os.path.join(folder_data, 'deces-{}.csv'.format(year))
        dfs_deces_raw.append(pd.read_csv(path, sep=';', usecols=['sexe', 'datenaiss', 'datedeces', 'lieudeces'], dtype={'lieudeces':'str'}))
    names = ['deces-2020-m01', 'Deces_2020_M02', 'Deces_2020_M03']
    for name in names:
        path = os.path.join(folder_data, name + '.csv')
        dfs_deces_raw.append(pd.read_csv(path, sep=';', usecols=['sexe', 'datenaiss', 'datedeces', 'lieudeces'], dtype={'lieudeces':'str'}))
        
    # treat data
    for df in dfs_deces_raw:
        df.loc[(df['datenaiss'] % 10000) == 0, 'datenaiss'] += 701   # les gens dont on ne connait pas la date de naissance -> 1er juillet
        df.loc[(df['datenaiss'] % 10000 - df['datenaiss'] % 100) == 0, 'datenaiss'] += 700
        df.loc[(df['datenaiss'] % 100) == 0, 'datenaiss'] += 15
        df['deces_date'] = pd.to_datetime(df['datedeces'], format='%Y%m%d', errors='coerce')
        df['naiss_date'] = pd.to_datetime(df['datenaiss'], format='%Y%m%d', errors='coerce')
        df['age'] = (pd.to_datetime(df['datedeces'], format='%Y', exact=False) - df['naiss_date']).transform(lambda d: d.days // 365) # age au premier janvier
        df['depdeces'] = df['lieudeces'].transform(departement)
        df['annee'] = pd.to_numeric(df['datedeces'].astype(str).str.slice(0, 4), errors='coerce')
        df['month'] = pd.to_numeric(df['datedeces'].astype(str).str.slice(4, 6), errors='coerce')
        df['jour_annee'] = df['deces_date'].transform(lambda date: (date-date.replace(month=1, day=1)).days)
        df.drop(columns=['lieudeces', 'datenaiss', 'datedeces', 'naiss_date'], inplace=True)
    df_deces9819 = pd.concat(dfs_deces_raw)
    df_deces9819 = df_deces9819[df_deces9819['deces_date'] <= datetime.datetime(2019, 12, 31)] # on supprime les mauvaises données pour 2020
    
    # save data 
    path = os.path.join(folder_treated_data, 'deces9819.pkl')
    df_deces9819.to_pickle(path)
    
else:
    # load data
    path = os.path.join(folder_treated_data, 'deces9819.pkl')
    df_deces9819 = pd.read_pickle(path)

In [ ]:
df_deces9819.head()

### Données détaillées sur les décès Janv 18 -Avril 20 

In [ ]:
path = os.path.join(folder_data, 'DC_jan2018-avr2020_det.csv')
df_detail_deces1820 = pd.read_csv(path, sep=';')
for i in range(1, 10):
    df_detail_deces1820['DEPDEC'].replace(str(i), '0' + str(i), inplace=True)
    df_detail_deces1820['DEPDOM'].replace(str(i), '0' + str(i), inplace=True)
    df_detail_deces1820['MDEC'].replace(i, '0' + str(i), inplace=True)
    df_detail_deces1820['JDEC'].replace(i, '0' + str(i), inplace=True)
df_detail_deces1820['SEXE'].replace('M', 1, inplace=True)
df_detail_deces1820['SEXE'].replace('F', 2, inplace=True)
df_detail_deces1820['age'] = (df_detail_deces1820['ADEC']-1-df_detail_deces1820['ANAIS']).astype('float64') #.transform(batch_age)
df_detail_deces1820['deces_date'] = df_detail_deces1820['ADEC'].astype('str') + df_detail_deces1820['MDEC'].astype('str') + df_detail_deces1820['JDEC'].astype('str')
df_detail_deces1820['deces_date'] = pd.to_datetime(df_detail_deces1820['deces_date'], format='%Y%m%d')
df_detail_deces1820['jour_annee'] = df_detail_deces1820['deces_date'].transform(lambda date: (date-date.replace(month=1, day=1)).days)

In [ ]:
df_detail_deces1820 = df_detail_deces1820.drop(['JDEC', 'ANAIS', 'MNAIS', 'JNAIS'], axis=1)
df_detail_deces1820.columns = ['annee', 'month', 'depdeces', 'comdeces', 'sexe', 'depdom', 'lieudeces', 'age', 'deces_date', 'jour_annee']
df_detail_deces1820 = df_detail_deces1820.reindex(columns=['sexe', 'deces_date', 'age', 'depdeces', 'annee', 'month', 'jour_annee', 'comdeces', 'lieudeces', 'depdom'])

In [ ]:
df_detail_deces1820.head()

In [ ]:
path = os.path.join(folder_treated_data, 'detail_deces1820.pkl')
df_detail_deces1820.drop(['annee', 'month', 'jour_annee'], axis=1).to_pickle(path)

### Ajout des décès en 2020

L'INSEE précise que les bonnes données de décès pour 2020 sont contenues dans le fichier exceptionnel 'donnees_insee/DC_Jan-Avr_2018-2020_det.csv' et pas dans les fichiers usuels de décès utilisés précédemment.

In [ ]:
df_2020 = df_detail_deces1820.loc[df_detail_deces1820['annee'] == 2020]
df_2020.drop(columns=['comdeces', 'lieudeces', 'depdom'], inplace=True)
df_deces9820 = pd.concat([df_deces9819, df_2020])   # on rajoute les bonnes

In [ ]:
df_deces9820 = df_deces9820[df_deces9820['age'] >= 20]
df_deces9820 = df_deces9820[df_deces9820['annee'] >= 1998]
df_deces9820 = df_deces9820.reset_index(drop=True)

In [ ]:
df_deces9820.head()

In [ ]:
path = os.path.join(folder_treated_data, 'deces9820.pkl')
df_deces9820.to_pickle(path)

### Population par âge et année 

In [ ]:
path = os.path.join(folder_data, 'fe_t6.xls')
donnees_fe_t6_1 = pd.read_excel(path, sheet_name=list(map(str, range(1991, 1999))), skiprows=8, header=0, index_col=1)

In [ ]:
donnees_fe_t6_1_final = {}
for annee in range(1991, 1999):
    donnees_fe_t6_1[str(annee)] = donnees_fe_t6_1[str(annee)].iloc[20:101,1:]
    donnees_fe_t6_1[str(annee)].columns = ['Ensemble', 1, 2]
    donnees_fe_t6_1[str(annee)].index = list(range(20, 101))
    donnees_fe_t6_1_final[annee] = donnees_fe_t6_1[str(annee)].T

In [ ]:
donnees_fe_t6_2 = pd.read_excel(path, sheet_name=list(map(str, range(1999, 2006))), skiprows=8, header=0, index_col=1)

In [ ]:
donnees_fe_t6_2_final = {}
for annee in range(1999, 2006):
    donnees_fe_t6_2[str(annee)] = donnees_fe_t6_2[str(annee)].iloc[20:106,1:]
    donnees_fe_t6_2[str(annee)].columns = ['Ensemble', 1, 2]
    donnees_fe_t6_2[str(annee)].loc['100 '] = np.sum(donnees_fe_t6_2[str(annee)].iloc[-6:])
    donnees_fe_t6_2[str(annee)] = donnees_fe_t6_2[str(annee)].drop(donnees_fe_t6_2[str(annee)].index[-5:])
    donnees_fe_t6_2[str(annee)].index = list(range(20, 101))
    donnees_fe_t6_2_final[annee] = donnees_fe_t6_2[str(annee)].T

In [ ]:
donnees_fe_t6_3 = pd.read_excel(path, sheet_name=list(map(str, range(2006, 2013))), skiprows=8, index_col=1)

In [ ]:
donnees_fe_t6_3_final = {}
for annee in range(2006, 2013):
    donnees_fe_t6_3[str(annee)] = donnees_fe_t6_3[str(annee)].iloc[20:106, [1, 2, 7]]
    donnees_fe_t6_3[str(annee)].columns = ['Ensemble', 1, 2]
    donnees_fe_t6_3[str(annee)].loc['100 '] = np.sum(donnees_fe_t6_3[str(annee)].iloc[-6:])
    donnees_fe_t6_3[str(annee)] = donnees_fe_t6_3[str(annee)].drop(donnees_fe_t6_3[str(annee)].index[-5:])
    donnees_fe_t6_3[str(annee)].index = list(range(20, 101))
    donnees_fe_t6_3_final[annee] = donnees_fe_t6_3[str(annee)].T

In [ ]:
donnees_fe_t6_4 = pd.read_excel(path, sheet_name=list(map(str, range(2013, 2018))), skiprows=3, index_col=1)

In [ ]:
donnees_fe_t6_4_final = {}
for annee in range(2013, 2018):
    donnees_fe_t6_4[str(annee)] = donnees_fe_t6_4[str(annee)].iloc[20:106, [1, 2, 7]]
    donnees_fe_t6_4[str(annee)].columns = ['Ensemble', 1, 2]
    donnees_fe_t6_4[str(annee)].loc[100] = np.sum(donnees_fe_t6_4[str(annee)].iloc[-6:])
    donnees_fe_t6_4[str(annee)] = donnees_fe_t6_4[str(annee)].drop(donnees_fe_t6_4[str(annee)].index[-5:])
    donnees_fe_t6_4[str(annee)].index = list(range(20, 101))
    donnees_fe_t6_4_final[annee] = donnees_fe_t6_4[str(annee)].T

In [ ]:
path = os.path.join(folder_data, 'T18F032.xls')
donnees_T18F032 = pd.read_excel(path, sheet_name=0, skiprows=3, index_col=1)
donnees_T18F032 = donnees_T18F032.iloc[20:101, [2, 4]]
donnees_T18F032.insert(0, 'Ensemble', donnees_T18F032['Hommes France entière'] + donnees_T18F032['Femmes France entière'])
donnees_T18F032.index = list(range(20, 101))
donnees_T18F032.columns = [(2018, 'Ensemble'), (2018, 1), (2018, 2)]
donnees_T18F032 = donnees_T18F032.T

In [ ]:
path = os.path.join(folder_data, 'T19F032.xls')
donnees_T19F032 = pd.read_excel(path, sheet_name=0, skiprows=3, index_col=1)
donnees_T19F032 = donnees_T19F032.iloc[20:101, [2, 4]]
donnees_T19F032.insert(0, 'Ensemble', donnees_T19F032['Hommes France entière'] + donnees_T19F032['Femmes France entière'])
donnees_T19F032.index = list(range(20, 101))
donnees_T19F032.columns = [(2019, 'Ensemble'), (2019, 1), (2019, 2)]
donnees_T19F032 = donnees_T19F032.T

In [ ]:
path = os.path.join(folder_data, 'pop-totale-france.xls')
donnees_pop_20 = pd.read_excel(path, sheet_name=0, skiprows=5, index_col=1)
donnees_pop_20 = donnees_pop_20.iloc[20:101, [1, 2]]
donnees_pop_20.insert(0, 'Ensemble', donnees_pop_20["Nombre d'hommes"] + donnees_pop_20['Nombre de femmes'])
donnees_pop_20.index = list(range(20, 101))
donnees_pop_20.columns = [(2020, 'Ensemble'), (2020, 1), (2020, 2)]
donnees_pop_20 = donnees_pop_20.T

In [ ]:
df_pop9820 = pd.concat([pd.concat(donnees_fe_t6_1_final, sort=False), 
                pd.concat(donnees_fe_t6_2_final, sort=False),
                pd.concat(donnees_fe_t6_3_final, sort=False),
                pd.concat(donnees_fe_t6_4_final, sort=False),
                10**3 * donnees_T18F032,       # 10**3 à cause du sép décimal différent dans les fichiers INSEE
                10**3 * donnees_T19F032,
               donnees_pop_20])

In [ ]:
df_pop9820 = df_pop9820.swaplevel().sort_index()

In [ ]:
path = os.path.join(folder_treated_data, 'population_age_sexe_9820.pkl')
df_pop9820.to_pickle(path)

### Population par classe d'âge et département 

In [ ]:
path = os.path.join(folder_data, 'estim-pop-dep-sexe-aq-1975-2020.xls')
df = pd.read_excel(path, sheet_name=list(map(str, range(1999, 2021))), skiprows=3)
pop = {}

In [ ]:
def transform(df):
    col = list(df.iloc[0])
    col[0] = 'Département'
    df.columns = col
    df = df.drop(index=0)
    df.index = df['Département']
    return df.drop(columns='Département')

In [ ]:
for annee in range(1999, 2021):
    pop[annee] = {}
    df_H = df[str(annee)].iloc[:97, [0] + list(range(27,43))]
    df_F = df[str(annee)].iloc[:97, [0] + list(range(48,64))]
    pop[annee][1] = transform(df_H)
    pop[annee][2] = transform(df_F)
    pop[annee] = pd.concat(pop[annee].values(), keys=pop[annee].keys(), names=['Sexe'])

In [ ]:
pop = pd.concat(pop.values(), keys=pop.keys(), names=['Année']).swaplevel(i=-3, j=-1).sort_index()

In [ ]:
pop

In [ ]:
path = os.path.join(folder_treated_data, 'population_dep_classeage_sexe_9920.pkl')
pop.to_pickle(path)

### Décès en milieu hospitalier et en EHPAD 

In [ ]:
path = os.path.join(folder_data, 'deces_hop_ehpad.csv')
donnees_hop_ephad = pd.read_csv(path, sep=';', usecols=['jour', 'dc off'])

In [ ]:
donnees_hop_ephad = donnees_hop_ephad.loc[~donnees_hop_ephad['dc off'].isna()]
donnees_hop_ephad['jour'] = pd.to_datetime(donnees_hop_ephad['jour'], format='%d/%m/%Y')
donnees_hop_ephad.index = donnees_hop_ephad['jour']
donnees_hop_ephad.drop(['jour'], axis=1, inplace=True)

In [ ]:
donnees_hop_ephad.head()

In [ ]:
path = os.path.join(folder_treated_data, 'deces_hop_ehpad.pkl')
donnees_hop_ephad.to_pickle(path)